<a href="https://colab.research.google.com/github/jihyunjeongme/deeplearning-tensflow-3min/blob/master/%5B3%EB%B6%84_%EB%94%A5%EB%9F%AC%EB%8B%9D%5D_7%EC%9E%A5_%EC%9D%B4%EB%AF%B8%EC%A7%80_%EC%9D%B8%EC%8B%9D%EC%9D%98_%EC%9D%80%EC%B4%9D%EC%95%8C_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 합성곱 신경망(CNN)

---

CNN(Convolutional Neural Network)은 1998년 얀 레쿤 교수가 소개한 이래로 널리 사용되고 있는 신경망.

# 7.1 CNN 개념

---

- 컨볼루션 계층(합성곱 계층)과 풀링 계층으로 구성
- 컨볼루션(2D): 2차원의 평면 행렬에서 지정한 영역의 값들을 하나의 값으로 압축하는 것.
  - 단 하나의 값으로 압축할 때 컨볼루션 계층은 가중치와 편향을 적용하고, 풀링계층은 단순히 값들 중 하나를 선택해서 가져오는 방식
- 윈도우: 지정한 크기의 영역
- 스트라이드: 몇 칸씩 움직일지 정하는 값
- 입력층의 윈도우를 은닉층의 뉴런 하나로 압축할 떄, 컨볼루션 계층에서는 윈도우 크기만큼의 가중치와 1개의 편향을 적용합니다.
  - ex) 윈도우 크기가 3 x 3 이라면, 3 x 3개의 가중치와 1개의 편향이 필요합니다.
  - 이 3 x 3개의 가중치와 1개의 편향을 커널 또는 필터 라고 하며
  - 이 커널은 해당 은닉층을 만들기 위한 모든 윈도우에 공통으로 적용됩니다.

# 7.2 모델 구현하기

---

앞 장에서 사용한 MNIST 데이터를 CNN으로 학습시키는 모델을 만들어보면서 조금 더 자세히 알아보도록 하겠습니다.

In [48]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [49]:
# 앞 장에서 만든 모델에서는 입력값을 28 x 28짜리 하나로 구성했지만, CNN 모델에서는 
# 앞서 설명한 것 처럼 2차원 평면으로 구성하므로 다음처럼 조금 더 직관적인 형태로 구성할수 있음.


# X의 첫 번째 차원인 None은 입력 데이터의 개수. 마지막 차원인 1은 특징의 개수
# MINIST 데이터는 회색조 이미지라 채널에 색상이 한개 뿐이므로 1을 사용
X = tf.placeholder(tf.float32, [None, 28, 28, 1])

# 출력값인 10개의 분류와, 드롭아웃을 위한 keep_prob 플레이스 홀더도 정의
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

print(X)

Tensor("Placeholder_18:0", shape=(?, 28, 28, 1), dtype=float32)


In [50]:
# CNN 계층을 구성
# 3 x 3 크기의 커널을 가진 컨볼루션 계층을 만듬
# tf.nn.conv2d 함수를 사용하면 간단하게 구성할 수 있음.

W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1,1,1,1], padding='SAME')
L1 = tf.nn.relu(L1)

# 28 x 28 ----> 32개
print(L1)

Tensor("Relu_8:0", shape=(?, 28, 28, 32), dtype=float32)


In [0]:
# 입력층 X와 첫 번째 계층의 가중치 W1을 가지고, 오른쪽과 아래쪽 한칸씩 움직이는 32개의 커널을 가진
# 컨볼루션 계층을 만들겠다는 코드
# padding='SAME'은 커널 슬라이딩 시 이미지의 가장 외곽에서 한 칸 밖으로 움직이는 옵션
# 이렇게 하면 이미지의 테두리까지도 조금 더 정확하게 평가 할 수 있습니다.
# tf.nn.relu 활성화 함수를 통해 컨볼루션 계층 완성

In [52]:
# 풀링계층 만듬

L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1],
                    padding='SAME')

# 14 x 14 ----> 32개
# 풀링계층에서 갯수가 줄음
print(L1)

Tensor("MaxPool_5:0", shape=(?, 14, 14, 32), dtype=float32)


In [0]:
# 앞서 만든 컨볼루션 계층을 입력층으로 사용하고, 커널 크기를 2 x 2 로 하는 풀링 계층을 만듬
# strides=[1,2,2,1] 값은 슬라이딩 시 두칸씩 움직이겠다는 옵션

In [0]:
# 두번째 계층을 동일한 방식으로 구성
# 3 x 3 크기의 커널 64개로 구성한 컬볼루션 계층과
# 3, 3, 32, 64 여기에서 32는 앞 서 구성한 첫 번째 컨볼루션 계층의 커널 개수
# 이것은 출력층의 개수이며 또한 첫 번째 컨볼루션 계층이 찾아낸 이미지의 특징 개수


W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')
L2 = tf.nn.relu(L2)

# 2 x 2 크기의 풀링 계층으로 구성
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1],
                    padding='SAME')

In [0]:
# 이제 추출한 특징들을 이용해 10개의 분류를 만들어내는 계층을 구성해보겠습니다.

W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)

In [0]:
# 먼저 10개의 분류는 1차원 배열이므로 차원을 줄이는 단계를 거쳐야 합니다.
# 직전 풀링 계층 크기가 7 x 7 x 64 이므로 tf.reshpae 함수를 이용해 7 x 7 x 64 크기의 1차원
# 계층으로 만들고, 이 배열 전체를 최종 출력값의 중간 단계 인 256개의 뉴런으로 연결하는 신경망을 만듬.

# 이처럼 인접한 계층의 모든 뉴런과 상호 연결된 계층을 '완전연결계층' 이라고 함.
# 이번 계층에서는 과적합을 막아주는 드롭아웃 기법 사용

In [57]:
# 모델 구성의 마지막으로 직전의 은닉층인 L3의 출력값 256개를 받아 최종출력값인 0~9 레이블을 갖는
# 10개의 출력값을 만듭니다.

W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)
print(W4)
print(model)

<tf.Variable 'Variable_17:0' shape=(256, 10) dtype=float32_ref>
Tensor("MatMul_7:0", shape=(?, 10), dtype=float32)


In [0]:
# 손실 함수와 AdamOptimizer를 이용한 최적화 함수를 만듭니다.

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
                        logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [0]:
# 나중에 최적화 함수를 RMSPropOptimizer로 바꿔서 결과를 비교해보는 것도 좋을 듯

# optimizer = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)

In [62]:
# 이제 학습 시키고 결과를 확인 하는 코드 작성
# 모델에 입력값을 전달하기 위해 MNIST 데이터를 28 x 28 형태로 재구성하는 부분을 아래와 같이 작성

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(50):
  total_cost = 0
  
  for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    batch_xs = batch_xs.reshape(-1, 28, 28, 1)
  
    _, cost_val = sess.run([optimizer, cost],
                            feed_dict={X: batch_xs,
                                       Y: batch_ys,
                                       keep_prob: 0.7})
    
    total_cost += cost_val
  
  print('Epoch:', '%04d' % (epoch + 1),
        'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))

print('최적화 완료!')


is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

print('정확도:', sess.run(accuracy,
                         feed_dict={X: mnist.test.images.reshape(
                                      -1, 28, 28, 1),
                                    Y: mnist.test.labels,
                                    keep_prob: 1}))

Epoch: 0001 Avg. cost = 0.342
Epoch: 0002 Avg. cost = 0.109
Epoch: 0003 Avg. cost = 0.076
Epoch: 0004 Avg. cost = 0.059
Epoch: 0005 Avg. cost = 0.050
Epoch: 0006 Avg. cost = 0.042
Epoch: 0007 Avg. cost = 0.037
Epoch: 0008 Avg. cost = 0.030
Epoch: 0009 Avg. cost = 0.028
Epoch: 0010 Avg. cost = 0.026
Epoch: 0011 Avg. cost = 0.023
Epoch: 0012 Avg. cost = 0.019
Epoch: 0013 Avg. cost = 0.019
Epoch: 0014 Avg. cost = 0.017
Epoch: 0015 Avg. cost = 0.017
Epoch: 0016 Avg. cost = 0.014
Epoch: 0017 Avg. cost = 0.014
Epoch: 0018 Avg. cost = 0.012
Epoch: 0019 Avg. cost = 0.011
Epoch: 0020 Avg. cost = 0.010
Epoch: 0021 Avg. cost = 0.010
Epoch: 0022 Avg. cost = 0.010
Epoch: 0023 Avg. cost = 0.009
Epoch: 0024 Avg. cost = 0.009
Epoch: 0025 Avg. cost = 0.009
Epoch: 0026 Avg. cost = 0.007
Epoch: 0027 Avg. cost = 0.008
Epoch: 0028 Avg. cost = 0.007
Epoch: 0029 Avg. cost = 0.007
Epoch: 0030 Avg. cost = 0.006
Epoch: 0031 Avg. cost = 0.008
Epoch: 0032 Avg. cost = 0.005
Epoch: 0033 Avg. cost = 0.005
Epoch: 003

# 7.3 고수준 API

---

layers 모듈을 이용하여 앞서 만든 CNN 모델을 조금 더 간단하게 만들기